# Bryan Flood 
## Profiling the 1994 US Census with Pandas

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

### Read in dataset and split into continuous and categorical

In [2]:
feature_names_text_file = open("./data/feature_names.txt", "r")
feature_names = feature_names_text_file.read().split("\n")

In [3]:
dataset = pd.read_csv("./data/dataset.csv",
                          sep=',',
                          names=feature_names,
                          na_values=[' ?', ' 0'])
dataset = dataset.drop(columns="id")

In [4]:
continuous = dataset.select_dtypes(include=[np.number])
categorical = dataset.select_dtypes(exclude=[np.number])

## Continuous Features Quality Report

In [5]:
continuous_count           = pd.DataFrame(continuous.count(),
                                   columns=['Count'])
continuous_missing_count   = pd.DataFrame(continuous.isna().sum(),
                                   columns=['Count'])

continuous_sum = continuous_count.add(continuous_missing_count, fill_value=0)

continuous_percent_miss    = pd.DataFrame(columns=['percent miss'])
for cell in list(continuous.columns.values):
    continuous_percent_miss.loc[cell] = continuous_missing_count.loc[cell].values / continuous_sum.loc[cell].values * 100

continuous_cardinality     = pd.DataFrame(columns=['cardinality'])
for cell in list(continuous.columns.values):
    continuous_cardinality.loc[cell] = [continuous[cell].nunique()]

continuous_minimum         = pd.DataFrame(columns=['minimum-value'])
for cell in list(continuous.columns.values):
    continuous_minimum.loc[cell] = [continuous[cell].min()]

continuous_first_quartile  = continuous.quantile([.25]).transpose()


continuous_mean            = continuous.mean().transpose()
continuous_median          = continuous.median().transpose()
continuous_third_quartile  = continuous.quantile([.75]).transpose()

continuous_maximum = pd.DataFrame(columns=['Maximum Value'])
for cell in list(continuous.columns.values):
    continuous_maximum.loc[cell] = [continuous[cell].max()]

continuous_std = continuous.std().transpose()

continuous_data_quality_report = pd.DataFrame()
continuous_data_quality_report['Count']           = continuous_sum['Count']
continuous_data_quality_report['% Miss.']         = continuous_percent_miss['percent miss']
continuous_data_quality_report['Card.']           = continuous_cardinality['cardinality']
continuous_data_quality_report['Min.']            = continuous_minimum['minimum-value']
continuous_data_quality_report['1st Qrt.']        = continuous_first_quartile[0.25]
continuous_data_quality_report['Mean']            = continuous_mean
continuous_data_quality_report['Median']          = continuous_median
continuous_data_quality_report['3rd Qrt.']        = continuous_third_quartile[0.75]
continuous_data_quality_report['Max.']            = continuous_maximum['Maximum Value']
continuous_data_quality_report['Std Dev.']        = continuous_std

continuous_data_quality_report

,Count,% Miss.,Card.,Min.,1st Qrt.,Mean,Median,3rd Qrt.,Max.,Std Dev.
age,30940,0.000000,72,17,28.0,38.560763,37.0,48.0,90,13.639403
fnlwgt,30940,0.000000,20880,12285,117849.0,189786.401422,178384.0,237318.0,1484705,105406.394386
education-num,30940,0.000000,16,1,9.0,10.081254,10.0,12.0,16,2.569967
capital-gain,30940,91.735617,118,114,3411.0,13090.064138,7298.0,14084.0,99999,22659.543638
capital-loss,30940,95.378151,90,155,1672.0,1873.059441,1887.0,1977.0,4356,381.049337
hours-per-week,30940,0.000000,93,1,40.0,40.408920,40.0,45.0,99,12.336945


## Categorical Features Quality Report

In [6]:
categorical_count = pd.DataFrame(categorical.count(),
                                   columns=['Count'])

categorical_missing_count = pd.DataFrame(categorical.isna().sum(),
                                   columns=['Count'])
categorical_sum = categorical_count.add(categorical_missing_count, fill_value=0)

categorical_percent_miss    = pd.DataFrame(columns=['percent miss'])
for cell in list(categorical.columns.values):
    categorical_percent_miss.loc[cell] = categorical_missing_count.loc[cell].values / categorical_count.loc[cell].values * 100


categorical_cardinality     = pd.DataFrame(columns=['cardinality'])
for cell in list(categorical.columns.values):
    categorical_cardinality.loc[cell] = [categorical[cell].nunique()]
    
categorical_mode            = categorical.mode().transpose()


categorical_mode_freq = pd.DataFrame(columns=['Mode Freq'])
for cell in list(categorical.columns.values):
    categorical_mode_freq.loc[cell] = categorical[cell].value_counts()[0]
    
categorical_mode_percent = pd.DataFrame(columns=['1st Mode percent'])
for cell in list(categorical.columns.values):
    categorical_mode_percent.loc[cell] = (categorical[cell].value_counts()[0]/categorical_count.loc[cell].values) * 100


categorical_second_mode = pd.DataFrame(columns=['2nd Mode'])
for cell in list(categorical.columns.values):
    categorical_second_mode.loc[cell] = categorical[cell].value_counts().index[1]


categorical_second_mode_freq = pd.DataFrame(columns=['2nd Mode freq'])
for cell in list(categorical.columns.values):
    categorical_second_mode_freq.loc[cell] = categorical[cell].value_counts()[1]

categorical_second_mode_percent = pd.DataFrame(columns=['2nd Mode percent'])
for cell in list(categorical.columns.values):
    categorical_second_mode_percent.loc[cell] = (categorical[cell].value_counts()[1]/categorical_count.loc[cell].values) * 100



categorical_data_quality_report = pd.DataFrame()
categorical_data_quality_report['Count']          = categorical_sum['Count']
categorical_data_quality_report['% Miss']         = categorical_percent_miss
categorical_data_quality_report['Card.']          = categorical_cardinality['cardinality']
categorical_data_quality_report['Mode']           = categorical_mode
categorical_data_quality_report['Mode Freq']      = categorical_mode_freq
categorical_data_quality_report['Mode %']         = categorical_mode_percent
categorical_data_quality_report['2nd Mode']       = categorical_second_mode
categorical_data_quality_report['2nd Mode Freq']  = categorical_second_mode_freq
categorical_data_quality_report['2nd Mode %']     = categorical_second_mode_percent

categorical_data_quality_report

,Count,% Miss,Card.,Mode,Mode Freq,Mode %,2nd Mode,2nd Mode Freq,2nd Mode %
workclass,30940,5.940764,8,Private,21576,73.877761,Self-emp-not-inc,2406,8.238315
education,30940,0.000000,16,HS-grad,9976,32.243051,Some-college,6938,22.424047
marital-status,30940,0.000000,7,Married-civ-spouse,14201,45.898513,Never-married,10167,32.860375
occupation,30940,5.966162,14,Prof-specialty,3932,13.466676,Craft-repair,3887,13.312556
relationship,30940,0.000000,6,Husband,12496,40.387847,Not-in-family,7904,25.546218
race,30940,0.000000,5,White,26442,85.462185,Black,2965,9.583064
sex,30940,0.000000,2,Male,20705,66.919845,Female,10235,33.080155
native-country,30940,1.823208,41,United-States,27719,91.222932,Mexico,607,1.997630
target,30940,0.000000,2,<=50K,23506,75.972851,>50K,7434,24.027149


## Output Quality reports as csv

In [7]:
continuous_data_quality_report.to_csv("CONT.csv")
categorical_data_quality_report.to_csv("CAT.csv")